# Brain Tumor Classification


## Introduction

**In this notebook, we tackle the problem of brain tumor classification using deep learning. We aim to classify brain MRI images into three tumor types (Glioma Tumor,Memingioma Tumor, Pituitary Tumor) and normal scans.**

# Importing Libraries

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.layers import Conv2D,ConvLSTM2D, MaxPooling2D, Flatten, Dense, Dropout,MaxPooling2D, BatchNormalization, LSTM,Reshape
from tensorflow.keras.regularizers import l2

In [ ]:
tf.random.set_seed(4321)

## Data Loading and Preprocessing

**We start by loading and preprocessing the dataset. This includes data augmentation, resizing, and normalization.**

In [ ]:
batch_size = 32  
image_size = (224, 224)  # image size 

In [ ]:
datagen = ImageDataGenerator(
    rescale=1./255,        # Normalize pixel values to [0, 1]
    rotation_range=20,     # Randomly rotate images by up to 20 degrees
    width_shift_range=0.2, # Randomly shift images horizontally
    height_shift_range=0.2,# Randomly shift images vertically
    shear_range=0.2,       # Shear intensity
    zoom_range=0.2,        # Randomly zoom images
    horizontal_flip=True,  # Randomly flip images horizontally
    fill_mode='nearest'    # Fill any missing pixels with the nearest available
)

In [ ]:
train_data_gen = datagen.flow_from_directory(
    '/kaggle/input/brain-tumor-classification/Training',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_data_gen = datagen.flow_from_directory(
    '/kaggle/input/brain-tumor-classification/Testing',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
)

## Model Building

**We design and train a convolutional neural network (CNN) for brain tumor classification. The architecture includes multiple convolutional and pooling layers.**


In [ ]:
model = tf.keras.Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax') 
])

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)
# early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1)

In [ ]:
epochs = 20

history = model.fit(
    train_data_gen,
    epochs=epochs,
    validation_data=validation_data_gen,
    callbacks=[reduce_lr],
    verbose=1
)

model.save('brain_tumor_classification_model.h5')

## Evaluation

In [ ]:
test_loss, test_accuracy = model.evaluate(validation_data_gen, verbose=1)
print(f'Test Accuracy: {test_accuracy * 100:.6f}%')

## Conclusion

**In this project, we successfully built a deep learning model for brain tumor classification. This work has potential applications in medical imaging and diagnosis.**